In [46]:
#Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium

#Load your data
data = pd.read_csv('unnati_phase1_data_revised.csv')

In [3]:
print(data.isnull().sum())
# no null values found

Alert      0
Date       0
Time       0
Lat        0
Long       0
Vehicle    0
Speed      0
dtype: int64


In [5]:
duplicate_rows = data[data.duplicated()]

if not duplicate_rows.empty:
    print("Duplicate entries exist.")
else:
    print("No duplicate entries found.")


Duplicate entries exist.


In [47]:
# dropping duplicate entries
data.drop_duplicates(inplace=True)

In [48]:
# Define the speed bins
speed_bins = [0, 10, 20, 30, 40, 50, 60, 70]
speed_labels = [f"{i}-{j}" for i, j in zip(speed_bins[:-1], speed_bins[1:])]

# Create a new column 'Speed Range' based on the speed bins
data['Speed Range'] = pd.cut(data['Speed'], bins=speed_bins, labels=speed_labels)

# Convert 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Extract the month from the 'Date' column
data['Month'] = data['Date'].dt.month

# Group by 'Month,' 'Speed Range,' and 'Alert,' and count the occurrences
alert_frequency_by_month_speed = data.groupby(['Month', 'Speed Range', 'Alert']).size().reset_index(name='Frequency')

# Pivot the table to have Alert types as columns
alert_frequency_pivot = alert_frequency_by_month_speed.pivot_table(index=['Month', 'Speed Range'], columns='Alert', values='Frequency', fill_value=0).reset_index()

# Add a 'max' column that contains the name of the alert that is maximum in each category
alert_frequency_pivot['max'] = alert_frequency_pivot[['cas_fcw', 'cas_pcw', 'cas_ldw', 'cas_hmw']].idxmax(axis=1)

# Add a 'Total Alerts' column to calculate the total number of alerts generated in each category
alert_frequency_pivot['Total Alerts'] = alert_frequency_pivot[['cas_fcw', 'cas_pcw', 'cas_ldw', 'cas_hmw']].sum(axis=1)

pd.set_option('display.max_colwidth', 50)
# Display the result
print(alert_frequency_pivot.to_string(index=False))

 Month Speed Range  cas_fcw  cas_hmw  cas_ldw  cas_pcw     max  Total Alerts
     6        0-10        7      147       23      169 cas_pcw           346
     6       10-20       19      479       51      243 cas_hmw           792
     6       20-30       51      964       95      184 cas_hmw          1294
     6       30-40       98     1387      205       65 cas_hmw          1755
     6       40-50       56     1331      589       15 cas_hmw          1991
     6       50-60       43      975     2592       23 cas_ldw          3633
     6       60-70        2       17       52        1 cas_ldw            72
     7        0-10        1       18        3        5 cas_hmw            27
     7       10-20        1       40        4       18 cas_hmw            63
     7       20-30        2       61        4       14 cas_hmw            81
     7       30-40        7       71       11        4 cas_hmw            93
     7       40-50        7       58       20        1 cas_hmw            86

In [49]:
# frequency of alerts by month
alerts_by_month = alert_frequency_by_month_speed.groupby('Month')['Frequency'].sum().reset_index()

# Display the result
print(alerts_by_month)

   Month  Frequency
0      6       9883
1      7        504
2      8       9602


In [50]:
# Group by 'Month,' 'Speed Range,' and 'Alert,' and count the occurrences
alert_frequency_by_month_speed = data.groupby(['Month', 'Speed Range'])['Alert'].count().reset_index()

# Rename the count column for clarity
alert_frequency_by_month_speed.rename(columns={'Alert': 'Frequency'}, inplace=True)

# Display the result
print(alert_frequency_by_month_speed)


    Month Speed Range  Frequency
0       6        0-10        346
1       6       10-20        792
2       6       20-30       1294
3       6       30-40       1755
4       6       40-50       1991
5       6       50-60       3633
6       6       60-70         72
7       7        0-10         27
8       7       10-20         63
9       7       20-30         81
10      7       30-40         93
11      7       40-50         86
12      7       50-60        149
13      7       60-70          5
14      8        0-10        332
15      8       10-20        810
16      8       20-30       1574
17      8       30-40       1901
18      8       40-50       2071
19      8       50-60       2859
20      8       60-70         55


In [51]:
# Group by 'Speed Range' and count the occurrences of alerts
alerts_by_speed_range = data.groupby('Speed Range')['Alert'].count().reset_index()

# Rename the count column for clarity
alerts_by_speed_range.rename(columns={'Alert': 'Frequency'}, inplace=True)

# Display the result
print(alerts_by_speed_range)

# Sort the DataFrame by 'Frequency' in decreasing order
alerts_by_speed_range_sorted = alerts_by_speed_range.sort_values(by='Frequency', ascending=False)

# Display the sorted result
print(alerts_by_speed_range_sorted)


  Speed Range  Frequency
0        0-10        705
1       10-20       1665
2       20-30       2949
3       30-40       3749
4       40-50       4148
5       50-60       6641
6       60-70        132
  Speed Range  Frequency
5       50-60       6641
4       40-50       4148
3       30-40       3749
2       20-30       2949
1       10-20       1665
0        0-10        705
6       60-70        132


In [52]:
# Convert 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Extract the month from the 'Date' column
data['Month'] = data['Date'].dt.month

# Group by 'Month' and calculate the average speed for each month
average_speed_by_month = data.groupby('Month')['Speed'].mean().reset_index()

# Display the result
print(average_speed_by_month)

   Month      Speed
0      6  39.067913
1      7  36.095057
2      8  38.343844


In [53]:
# Convert 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Extract the month from the 'Date' column
data['Month'] = data['Date'].dt.month

# Group by 'Month' and 'Vehicle' and calculate the average speed for each combination
average_speed_by_month_vehicle = data.groupby(['Month', 'Vehicle'])['Speed'].mean().reset_index()

# Display the result
print(average_speed_by_month_vehicle)

    Month  Vehicle      Speed
0       6      805  39.126661
1       6     1995  25.388889
2       6     2846  40.941873
3       6     3143  43.492754
4       6     5339  35.691006
5       7      805  31.938865
6       7     2846  47.333333
7       7     5339  32.872727
8       8      805  36.958709
9       8     2846  47.548922
10      8     3143  37.499449
11      8     5339  34.865227


In [54]:

# Convert 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Group by 'Vehicle' and calculate the average, maximum, and minimum speeds for each vehicle
vehicle_speed_summary = data.groupby('Vehicle')['Speed'].agg(['mean', 'max', 'min']).reset_index()
vehicle_speed_summary.columns = ['Vehicle', 'Average Speed', 'Max Speed', 'Min Speed']

# Display the result
print(vehicle_speed_summary)

   Vehicle  Average Speed  Max Speed  Min Speed
0      805      38.199493         63          0
1     1995      25.388889         58          0
2     2846      43.203818         64          0
3     3143      37.788535         61          0
4     5339      35.133029         65          0


In [55]:

# Define the speed bins
speed_bins = [0, 10, 20, 30, 40, 50, 60, 70]
speed_labels = [f"{i}-{j}" for i, j in zip(speed_bins[:-1], speed_bins[1:])]

# Create a new column 'Speed Range' based on the speed bins
data['Speed Range'] = pd.cut(data['Speed'], bins=speed_bins, labels=speed_labels)

# Convert 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Group by 'Year,' 'Month,' 'Day,' 'Speed Range,' and 'Alert,' and count the occurrences
alert_frequency_by_date_speed = data.groupby(['Date', 'Speed Range', 'Alert']).size().reset_index(name='Frequency')

# Pivot the table to have Alert types as columns
alert_frequency_pivot = alert_frequency_by_date_speed.pivot_table(index=['Date', 'Speed Range'], columns='Alert', values='Frequency', fill_value=0).reset_index()

# Add a 'max' column that contains the name of the alert that is maximum on each date
alert_frequency_pivot['max'] = alert_frequency_pivot[['cas_fcw', 'cas_pcw', 'cas_ldw', 'cas_hmw']].idxmax(axis=1)

# Display the result
print(alert_frequency_pivot)


Alert       Date Speed Range  cas_fcw  cas_hmw  cas_ldw  cas_pcw      max
0     2022-06-01        0-10        0        1        1        3  cas_pcw
1     2022-06-01       10-20        0       17        7        5  cas_hmw
2     2022-06-01       20-30        2       22        6        4  cas_hmw
3     2022-06-01       30-40        1       26       19        2  cas_hmw
4     2022-06-01       40-50        0       21       38        1  cas_ldw
..           ...         ...      ...      ...      ...      ...      ...
352   2022-08-31       20-30        1       33        0       11  cas_hmw
353   2022-08-31       30-40        5       49        0        2  cas_hmw
354   2022-08-31       40-50        2       47       19        1  cas_hmw
355   2022-08-31       50-60        0       30       34        0  cas_ldw
356   2022-08-31       60-70        0        1        1        0  cas_ldw

[357 rows x 7 columns]


maximum warning in each month in each speed range came out to be cas_pcw
minimum warning cas_fcw

In [56]:
data['Latitude'] = data['Lat'].round(2)
data['Longitude'] = data['Long'].round(2)
data

,Alert,Date,Time,Lat,Long,Vehicle,Speed,Speed Range,Month,Latitude,Longitude
0,cas_ldw,2022-06-01,05:36:36,12.887403,80.083412,2846,48,40-50,6,12.89,80.08
1,cas_ldw,2022-06-01,05:36:47,12.892586,80.085794,2846,58,50-60,6,12.89,80.09
2,cas_ldw,2022-06-01,05:37:04,12.894932,80.087564,2846,57,50-60,6,12.89,80.09
3,cas_ldw,2022-06-01,05:37:31,12.898267,80.090062,2846,49,40-50,6,12.90,80.09
4,cas_ldw,2022-06-01,05:38:08,12.901594,80.092538,2846,58,50-60,6,12.90,80.09
...,...,...,...,...,...,...,...,...,...,...,...
21320,cas_pcw,2022-08-31,18:12:29,13.070668,80.265539,5339,30,20-30,8,13.07,80.27
21321,cas_pcw,2022-08-31,18:13:01,13.071554,80.264616,5339,34,30-40,8,13.07,80.26
21322,cas_hmw,2022-08-31,18:15:05,13.071181,80.258729,5339,24,20-30,8,13.07,80.26
21323,cas_pcw,2022-08-31,18:16:35,13.067723,80.255065,5339,23,20-30,8,13.07,80.26


In [57]:
# Group by 'Latitude' and 'Longitude', count the alerts, and find the most common alert
location_alerts = data.groupby(['Latitude', 'Longitude'])['Alert'].agg(['count', lambda x: x.mode()[0]]).reset_index()

# Rename the columns for clarity
location_alerts.rename(columns={'count': 'Total Alerts', 'mode': 'Most Common Alert'}, inplace=True)

# Display the result
print(location_alerts)

     Latitude  Longitude  Total Alerts <lambda_0>
0       12.34      79.78             4    cas_ldw
1       12.35      79.77             1    cas_ldw
2       12.36      79.78             1    cas_ldw
3       12.37      79.78             1    cas_ldw
4       12.38      79.78             2    cas_ldw
..        ...        ...           ...        ...
312     13.14      80.30             1    cas_hmw
313     13.15      80.30             1    cas_fcw
314     13.17      80.24             9    cas_hmw
315     13.18      80.31             2    cas_hmw
316     13.18      80.32             1    cas_hmw

[317 rows x 4 columns]


In [58]:
# Sort the DataFrame by "Total Alerts" column in descending order
location_alerts_sorted = location_alerts.sort_values(by="Total Alerts", ascending=False)

# Display the result
print(location_alerts_sorted)
location_alerts_sorted.to_csv("location_alerts_sorted.csv")

     Latitude  Longitude  Total Alerts <lambda_0>
150     12.94      80.13           654    cas_hmw
158     12.95      80.14           605    cas_hmw
109     12.88      80.08           605    cas_hmw
128     12.92      80.11           600    cas_hmw
142     12.93      80.12           537    cas_hmw
..        ...        ...           ...        ...
86      12.83      80.13             1    cas_hmw
85      12.83      80.12             1    cas_pcw
81      12.82      80.17             1    cas_hmw
286     13.08      80.26             1    cas_hmw
316     13.18      80.32             1    cas_hmw

[317 rows x 4 columns]


In [59]:
# Assuming your DataFrame has a column named 'Alert' for alert types
alert_frequency = data['Alert'].value_counts().reset_index()

# Rename the columns for clarity
alert_frequency.columns = ['Alert', 'Frequency']

# Display the result
print(alert_frequency)


     Alert  Frequency
0  cas_hmw      11939
1  cas_ldw       6431
2  cas_pcw       1976
3  cas_fcw        590


In [62]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

# Create a geocoder instance
geolocator = Nominatim(user_agent="myGeocoder")

# Initialize empty lists to store location information
locations = []
for index, row in location_alerts_sorted.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Perform reverse geocoding with error handling and retries
    retries = 3
    while retries > 0:
        try:
            location = geolocator.reverse((latitude, longitude), exactly_one=True)
            if location:
                locations.append(location.address)
            else:
                locations.append("Location not found")
            break  # Break the retry loop if successful
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Geocoding request failed. Retrying... ({retries} retries left)")
            retries -= 1

# Add the location information to the DataFrame
location_alerts_sorted['Location'] = locations

# Display the DataFrame with location information
print(location_alerts_sorted)

     Latitude  Longitude  Total Alerts <lambda_0>  \
150     12.94      80.13           654    cas_hmw   
158     12.95      80.14           605    cas_hmw   
109     12.88      80.08           605    cas_hmw   
128     12.92      80.11           600    cas_hmw   
142     12.93      80.12           537    cas_hmw   
..        ...        ...           ...        ...   
86      12.83      80.13             1    cas_hmw   
85      12.83      80.12             1    cas_pcw   
81      12.82      80.17             1    cas_hmw   
286     13.08      80.26             1    cas_hmw   
316     13.18      80.32             1    cas_hmw   

                                              Location  
150  Government Hospital of Thoracic Medicine, kupp...  
158  Chromepet, Pallavaram, Chengalpattu District, ...  
109  Vandalur zoo, Grand Southern Trunk Road, Vanda...  
128  Irumbuliyur, Tambaram, Chengalpattu District, ...  
142  Gandhi Road, East Tambaram, Tambaram, Chengalp...  
..                   

In [ ]:
# Group by 'Latitude' and 'Longitude', count the alerts, and find the most common alert
location_alerts = data.groupby(['Latitude', 'Longitude'])['Alert'].agg(['count', lambda x: x.mode()[0]]).reset_index()

# Rename the columns for clarity
location_alerts.rename(columns={'count': 'Total Alerts', '<lambda>': 'Most Common Alert'}, inplace=True)

# Calculate the average speed for each group
location_avg_speed = data.groupby(['Latitude', 'Longitude'])['Speed'].mean().reset_index()

# Round the average speed to 3 decimal places
location_avg_speed['Average Speed'] = location_avg_speed['Speed'].round(3)

# Drop the 'Speed' column, as we now have the rounded average speed
location_avg_speed.drop(columns=['Speed'], inplace=True)

# Merge the two DataFrames based on 'Latitude' and 'Longitude'
result = pd.merge(location_alerts, location_avg_speed, on=['Latitude', 'Longitude'])

# Display the result
print(result.sort_values(by="Average Speed", ascending=False,inplace = True))
print (result)

In [64]:
location_alerts_sorted["Average_speed"] = result["Average Speed"]
location_alerts_sorted.rename(columns={'<lambda_0>': 'Most Common Alert'}, inplace=True)

,Latitude,Longitude,Total Alerts,Most Common Alert,Location,Average_speed
150,12.94,80.13,654,cas_hmw,"Government Hospital of Thoracic Medicine, kupp...",37.878
158,12.95,80.14,605,cas_hmw,"Chromepet, Pallavaram, Chengalpattu District, ...",36.089
109,12.88,80.08,605,cas_hmw,"Vandalur zoo, Grand Southern Trunk Road, Vanda...",24.357
128,12.92,80.11,600,cas_hmw,"Irumbuliyur, Tambaram, Chengalpattu District, ...",43.308
142,12.93,80.12,537,cas_hmw,"Gandhi Road, East Tambaram, Tambaram, Chengalp...",33.214
...,...,...,...,...,...,...
86,12.83,80.13,1,cas_hmw,"Kumizhi, Chengalpattu, Chengalpattu District, ...",40.000
85,12.83,80.12,1,cas_pcw,"Chengalpattu, Chengalpattu District, Tamil Nad...",13.000
81,12.82,80.17,1,cas_hmw,"Vandalur - Mambakkam - Kelambakkam Road, Kolap...",48.000
286,13.08,80.26,1,cas_hmw,"EVR Periyar Salai, Mahaveer Colony, CMWSSB Div...",36.000


In [43]:
data = pd.read_csv("location.csv")
# Assuming you have a DataFrame named data
print(data[["Latitude", 'Longitude', 'Location','Total Alerts']].head(10))


   Latitude  Longitude                                           Location  \
0     12.94      80.13  Government Hospital of Thoracic Medicine, kupp...   
1     12.95      80.14  Chromepet, Pallavaram, Chengalpattu District, ...   
2     12.88      80.08  Vandalur zoo, Grand Southern Trunk Road, Vanda...   
3     12.92      80.11  Irumbuliyur, Tambaram, Chengalpattu District, ...   
4     12.93      80.12  Gandhi Road, East Tambaram, Tambaram, Chengalp...   
5     12.91      80.10  Grand Southern Trunk Road, Peerkankaranai, Iru...   
6     12.99      80.18  CMWSSB Division 201, St.Thomas Mount-Pallavara...   
7     12.97      80.15  Alandur, Chennai District, Tamil Nadu, 600043,...   
8     12.90      80.09  Tambaram, Chengalpattu District, Tamil Nadu, 6...   
9     13.01      80.21  CMWSSB Division 170, St. Thomas Mount Cantonme...   

   Total Alerts  
0           654  
1           605  
2           605  
3           600  
4           537  
5           513  
6           453  
7       

In [5]:

# Find the rows where Latitude is approximately 12.88 and Longitude is approximately 80.08
location_data = data[(data['Latitude'].between(12.94, 12.95)) & (data['Longitude'].between(80.13, 80.14))]

# Display the location data
print(location_data)

     Unnamed: 0  Latitude  Longitude  Total Alerts Most Common Alert  \
0           150     12.94      80.13           654           cas_hmw   
1           158     12.95      80.14           605           cas_hmw   
128         151     12.94      80.14            28           cas_hmw   

                                              Location  Average_speed  
0    Government Hospital of Thoracic Medicine, kupp...         37.878  
1    Chromepet, Pallavaram, Chengalpattu District, ...         36.089  
128  Chitlapakkam, Tambaram, Chengalpattu District,...         48.286  


In [43]:
location_alerts_sorted["Location"][0]
#chengalpattu

'Government Hospital of Thoracic Medicine, kuppuswamy Street, Chitlapakkam, Tambaram, Chengalpattu District, Tamil Nadu, 600064, India'

In [67]:
location_alerts_sorted.to_csv("location.csv")

In [31]:

# Group by 'Vehicle ID' and count the frequency of ADAS events
vehicle_event_frequency = data.groupby('Vehicle')['Alert'].count().reset_index()

# Rename the columns for clarity
vehicle_event_frequency.rename(columns={'Alert': 'Event Frequency'}, inplace=True)

# Sort the DataFrame by 'Event Frequency' in descending order to see the vehicles with the most events first
vehicle_event_frequency.sort_values(by='Event Frequency', ascending=False, inplace=True)

# Display the result
print(vehicle_event_frequency)


   Vehicle  Event Frequency
0      805             6702
4     5339             5698
2     2846             5657
3     3143             2861
1     1995               18


In [80]:

# Convert 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Extract the month from the 'Date' column
data['Month'] = data['Date'].dt.month

# Group by 'Month' and 'Vehicle' and count the frequency of ADAS events
vehicle_event_frequency_monthly = data.groupby(['Month', 'Vehicle'])['Alert'].count().reset_index()

# Rename the columns for clarity
vehicle_event_frequency_monthly.rename(columns={'Alert': 'Event Frequency'}, inplace=True)

# Sort the DataFrame by 'Event Frequency' in descending order to see the vehicles with the most events first
vehicle_event_frequency_monthly.sort_values(by=['Month', 'Event Frequency'], ascending=[True, False], inplace=True)

# Display the result
print(vehicle_event_frequency_monthly)


    Month  Vehicle  Event Frequency
0       6      805             4366
2       6     2846             3716
4       6     5339             2246
3       6     3143              138
1       6     1995               18
5       7      805              229
7       7     5339              165
6       7     2846              132
11      8     5339             3287
10      8     3143             2723
8       8      805             2107
9       8     2846             1809


In [37]:
vehicle_1995_data = data[data['Vehicle'] == 1995]

# Display the rows for vehicle 1995
print(vehicle_1995_data)
#adas system is not detecting properly

        Alert        Date      Time        Lat       Long  Vehicle  Speed
276   cas_ldw  2022-06-01  08:20:47  13.006265  80.127376     1995     58
278   cas_hmw  2022-06-01  08:21:29  12.999845  80.125450     1995     58
279   cas_hmw  2022-06-01  08:24:06  12.984884  80.120986     1995     58
280   cas_ldw  2022-06-01  08:25:24  12.980213  80.120703     1995     58
281   cas_ldw  2022-06-01  08:25:46  12.975876  80.119640     1995     55
3078  cas_ldw  2022-06-08  08:17:28  13.011778  80.127567     1995     55
3088  cas_ldw  2022-06-08  08:24:45  12.951457  80.114937     1995     58
3089  cas_ldw  2022-06-08  08:25:13  12.950195  80.110670     1995     57
3091  cas_ldw  2022-06-08  08:25:52  12.946238  80.106785     1995      0
3349  cas_pcw  2022-06-09  07:32:43  13.169852  80.239483     1995      0
3354  cas_hmw  2022-06-09  07:35:00  13.169839  80.239495     1995      0
3355  cas_hmw  2022-06-09  07:35:05  13.169839  80.239495     1995      0
3356  cas_hmw  2022-06-09  07:35:15  1

In [62]:
import pandas as pd

# Assuming you have a DataFrame named df with columns including 'Speed'

# Filter rows where 'Speed' is equal to 0

zero = data[data['Speed'] == 0]

zero['Latitude'] = zero['Lat'].round(2)
zero['Longitude'] = zero['Long'].round(2)
zero

C:\Users\lenovo\AppData\Local\Temp\ipykernel_28276\2767438683.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero['Latitude'] = zero['Lat'].round(2)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_28276\2767438683.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero['Longitude'] = zero['Long'].round(2)


,Alert,Date,Time,Lat,Long,Vehicle,Speed,Latitude,Longitude
38,cas_hmw,2022-06-01,06:16:46,12.795046,80.020296,805,0,12.80,80.02
57,cas_ldw,2022-06-01,06:24:57,13.024305,80.228471,2846,0,13.02,80.23
124,cas_ldw,2022-06-01,07:08:32,12.516883,79.887983,805,0,12.52,79.89
134,cas_ldw,2022-06-01,07:12:53,12.543551,79.904835,805,0,12.54,79.90
198,cas_ldw,2022-06-01,07:47:18,12.697172,79.972959,805,0,12.70,79.97
...,...,...,...,...,...,...,...,...,...
21100,cas_hmw,2022-08-31,06:37:55,13.038943,80.246828,5339,0,13.04,80.25
21135,cas_pcw,2022-08-31,07:28:23,13.053098,80.272004,5339,0,13.05,80.27
21220,cas_ldw,2022-08-31,08:23:58,12.923263,80.120547,3143,0,12.92,80.12
21227,cas_hmw,2022-08-31,08:28:55,12.909345,80.099412,3143,0,12.91,80.10


In [64]:
data = zero
# Group by 'Latitude' and 'Longitude', count the alerts, and find the most common alert
location_alerts = data.groupby(['Latitude', 'Longitude'])['Alert'].agg(['count', lambda x: x.mode()[0]]).reset_index()

# Rename the columns for clarity
location_alerts.rename(columns={'count': 'Total Alerts', '<lambda>': 'Most Common Alert'}, inplace=True)

# Calculate the average speed for each group
location_avg_speed = data.groupby(['Latitude', 'Longitude'])['Speed'].mean().reset_index()

# Round the average speed to 3 decimal places
location_avg_speed['Average Speed'] = location_avg_speed['Speed'].round(3)

# Drop the 'Speed' column, as we now have the rounded average speed
location_avg_speed.drop(columns=['Speed'], inplace=True)

# Merge the two DataFrames based on 'Latitude' and 'Longitude'
result = pd.merge(location_alerts, location_avg_speed, on=['Latitude', 'Longitude'])

# Display the result
print(result.sort_values(by="Total Alerts", ascending=False,inplace = True))
print (result)

None
     Latitude  Longitude  Total Alerts <lambda_0>  Average Speed
111     13.06      80.25           112    cas_hmw            0.0
30      12.88      80.08            96    cas_hmw            0.0
109     13.05      80.27            33    cas_pcw            0.0
113     13.06      80.27            31    cas_pcw            0.0
93      13.02      80.23            23    cas_hmw            0.0
..        ...        ...           ...        ...            ...
29      12.87      80.10             1    cas_ldw            0.0
52      12.94      80.18             1    cas_pcw            0.0
98      13.03      80.25             1    cas_ldw            0.0
70      12.98      80.12             1    cas_pcw            0.0
60      12.96      80.09             1    cas_ldw            0.0

[127 rows x 5 columns]


In [74]:
# finding location for speed 0

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

# Create a geocoder instance
geolocator = Nominatim(user_agent="myGeocoder")

# Initialize empty lists to store location information
locations = []
for index, row in result.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Perform reverse geocoding with error handling and retries
    retries = 3
    while retries > 0:
        try:
            location = geolocator.reverse((latitude, longitude), exactly_one=True)
            if location:
                locations.append(location.address)
            else:
                locations.append("Location not found")
            break  # Break the retry loop if successful
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Geocoding request failed. Retrying... ({retries} retries left)")
            retries -= 1

# Add the location information to the DataFrame
result['Location'] = locations

# Display the DataFrame with location information
print(result)

     Latitude  Longitude  Total Alerts <lambda_0>  Average Speed  \
111     13.06      80.25           112    cas_hmw            0.0   
30      12.88      80.08            96    cas_hmw            0.0   
109     13.05      80.27            33    cas_pcw            0.0   
113     13.06      80.27            31    cas_pcw            0.0   
93      13.02      80.23            23    cas_hmw            0.0   
..        ...        ...           ...        ...            ...   
29      12.87      80.10             1    cas_ldw            0.0   
52      12.94      80.18             1    cas_pcw            0.0   
98      13.03      80.25             1    cas_ldw            0.0   
70      12.98      80.12             1    cas_pcw            0.0   
60      12.96      80.09             1    cas_ldw            0.0   

                                              Location  
111  Ward 111, Zone 9 Teynampet, Chennai, Chennai D...  
30   Vandalur zoo, Grand Southern Trunk Road, Vanda...  
109  Pertho 

In [79]:
result["Location"][1]

'Grand Southern Trunk Road, Maduranthakam, Chengalpattu District, Tamil Nadu, 603306, India'

In [70]:
# Filter rows where Latitude is 13.06 and Longitude is 80.25
filtered_result = data[(data['Latitude'] == 13.06) & (data['Longitude'] == 80.25)]

# Display the filtered result
print(filtered_result["Vehicle"].value_counts())

805     97
5339    14
2846     1
Name: Vehicle, dtype: int64


In [73]:
# Filter rows where 'Speed' is equal to 0
zero_speed_data = data[data['Speed'] == 0]

# Group by 'Vehicle' and count the occurrences of speed 0
frequency_of_speed_zero = zero_speed_data.groupby('Vehicle')['Speed'].count().reset_index()
frequency_of_speed_zero.rename(columns={'Speed': 'Frequency of Speed 0'}, inplace=True)

# Display the result
print(frequency_of_speed_zero)

   Vehicle  Frequency of Speed 0
0      805                   437
1     1995                    10
2     2846                   226
3     3143                   111
4     5339                   163


In [86]:
# Convert 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Round latitude and longitude to 2 decimal places
data['Latitude'] = data['Lat'].round(2)
data['Longitude'] = data['Long'].round(2)

# Extract the month from the 'Date' column
data['Month'] = data['Date'].dt.month

# Group by 'Month,' 'Latitude,' 'Longitude,' and count the occurrences of alerts
location_alerts_monthly = data.groupby(['Month', 'Latitude', 'Longitude'])['Alert'].count().reset_index()

# Rename the columns for clarity
location_alerts_monthly.rename(columns={'Alert': 'Alerts Generated'}, inplace=True)

# Sort the DataFrame by 'Month' in ascending order and 'Alerts Generated' in descending order
sorted_location_alerts = location_alerts_monthly.sort_values(by=['Month', 'Alerts Generated'], ascending=[True, False])

# Display the sorted result
print(sorted_location_alerts)

     Month  Latitude  Longitude  Alerts Generated
85       6     12.88      80.08               300
115      6     12.94      80.13               282
109      6     12.93      80.12               273
98       6     12.92      80.11               261
121      6     12.95      80.14               257
..     ...       ...        ...               ...
578      8     13.04      80.15                 1
582      8     13.05      80.08                 1
585      8     13.05      80.16                 1
600      8     13.06      80.28                 1
608      8     13.07      80.28                 1

[617 rows x 4 columns]


In [88]:
# Filter rows where 'Month' is equal to 6
filtered_rows_month_6 = sorted_location_alerts[sorted_location_alerts['Month'] == 6]

# Display the filtered result
print(filtered_rows_month_6.head(50))

     Month  Latitude  Longitude  Alerts Generated
85       6     12.88      80.08               300
115      6     12.94      80.13               282
109      6     12.93      80.12               273
98       6     12.92      80.11               261
121      6     12.95      80.14               257
95       6     12.91      80.10               222
139      6     12.97      80.15               218
159      6     13.00      80.20               218
153      6     12.99      80.18               210
158      6     13.00      80.19               200
163      6     13.01      80.21               197
164      6     13.01      80.22               187
91       6     12.90      80.09               182
147      6     12.98      80.16               158
193      6     13.05      80.27               152
179      6     13.03      80.24               151
148      6     12.98      80.17               146
213      6     13.07      80.27               142
174      6     13.02      80.23               136


In [90]:
# Filter rows where 'Month' is equal to 6
filtered_rows_month_6 = sorted_location_alerts[sorted_location_alerts['Month'] == 7]

# Display the filtered result
print(filtered_rows_month_6.head(50))

     Month  Latitude  Longitude  Alerts Generated
302      7     12.94      80.13                24
298      7     12.92      80.11                18
307      7     12.98      80.16                16
317      7     13.01      80.23                16
313      7     13.00      80.20                15
315      7     13.01      80.21                15
296      7     12.91      80.10                14
336      7     13.07      80.27                14
303      7     12.95      80.14                13
316      7     13.01      80.22                13
291      7     12.88      80.08                12
300      7     12.93      80.12                11
310      7     12.99      80.18                11
338      7     13.08      80.28                11
290      7     12.87      80.08                10
306      7     12.97      80.15                10
334      7     13.07      80.25                10
259      7     12.66      79.95                 9
332      7     13.06      80.26                 9


In [89]:
# Filter rows where 'Month' is equal to 6
filtered_rows_month_6 = sorted_location_alerts[sorted_location_alerts['Month'] == 8]

# Display the filtered result
print(filtered_rows_month_6.head(50))

     Month  Latitude  Longitude  Alerts Generated
493      8     12.94      80.13               348
499      8     12.95      80.14               335
471      8     12.92      80.11               321
454      8     12.88      80.08               293
466      8     12.91      80.10               277
462      8     12.90      80.09               255
485      8     12.93      80.12               253
540      8     12.99      80.18               232
518      8     12.97      80.15               224
553      8     13.01      80.22               214
552      8     13.01      80.21               210
548      8     13.00      80.20               185
528      8     12.98      80.16               177
457      8     12.89      80.08               176
547      8     13.00      80.19               172
599      8     13.06      80.27               166
509      8     12.96      80.15               163
607      8     13.07      80.27               154
508      8     12.96      80.14               152
